# Semi-supervised classification: Spam email recognition

En este notebook demostramos como podemos utilizar aprendizaje semi-supervisado para entrenar un modelo cuando tenemos pocas muestras con etiquetas.

Especificamente utilizaremos la técnica conocida como "self-labeling", la cuál utiliza un clasificador base para incrementar, de forma iterativa, las etiquetas por medio de usar las predicciones donde el clasificador se siente con mayor confianza y usarlas como etiquetas "reales" para la siguiente iteración.

La solución que presentamos a continuación esta basada en usar un algoritmo de la MLlib, es decir, es una solución "global".

In [ ]:
!pip install pyspark

In [ ]:
import numpy as np

from pyspark.sql import SparkSession
import pyspark.sql.functions as F

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.functions import vector_to_array

In [ ]:
spark = SparkSession.builder\
                    .master("local[*]")\
                    .appName("aprendizaje_semi_supervised")\
                    .getOrCreate()\

sc = spark.sparkContext

In [ ]:
#--otorgamos acceso a google drive
from google.colab import drive
drive.mount('/content/drive')

Utilizaremos el conjunto de datos __spambase.csv__ con el objetivo de entrenar un clasificador que pueda distinguir entre correos de genuinos y spam.

El conjunto de datos contiene __4601__ instnacias o filas, __57__ atributos y la variable respuesta.

Los atributos contienen entre otras cosas, una medida de repetición de palabras, longitud de palabras, etc. Todos los atributos son númericos.

La última columna contiene a la variable respuesta.

In [ ]:
PATH_INPUT = '/content/drive/MyDrive/data_sets/spambase.csv'

df = spark.read.format('csv')\
    .option("header", 'false')\
    .load(PATH_INPUT)

print(f'Número de instancias: {df.count()}')
print(f'Número de columnas: {len(df.columns)}')
df.show(3)

### Pre-procesamiento y división de nuestro conjunto de datos

In [ ]:
#--De momento, solo nos interesa tener control de la variable respuesta,
# por lo que la vamos a renombrar
df = df.withColumnRenamed("_c57", "output")

#--Dado que MLlib solo recive vectores de tipo float, convertimos todo
# a float de una vez
df = df.select([F.col(c).cast("float").alias(c) for c in df.columns])

In [ ]:
#--Dividimos nuestro datos en entrenamiento y prueba
seed = 12345
train, test = df.randomSplit([0.7, 0.3], seed)

Medimos cuantas instancias tenemos de cada clase:

In [ ]:
#--contamos las instancias por clase
stats = train\
    .groupBy("output")\
    .count()

#--calculamos el porcentaje
stats = stats\
    .select("*",
            F.round((stats["count"] / train.count() * 100), 2).alias("ratio(%)"),
    )

stats.show()

En este caso tenemos casi un 40% de la clase spam, además de que todas las instancias tienen etiqueta. Para utilizar el algoritmo de aprendizaje semi supervisado vamos a simular que tenemos pocas instancias con etiquetas.

Creamos una columna nueva que solo contenga un aproximado de 2% de etiquetas.

In [ ]:
#--definimos la porcentaje de etiquetas que deseamos
pct = 0.02

#--agregamos una nueva columna con solo el 2% de etiquetas.
# Para identificar muestras que no tiene etiquetas utilizamos el número 2
train = train\
    .withColumn("label",
                F.when(F.rand(seed=12345) < pct, train.output).otherwise(2),
    )

In [ ]:
train.sample(.1).show(3)

Revisamos la distribución de las etiquetas:

In [ ]:
stats = train\
    .groupBy("label")\
    .count()

stats = stats.select("*",
    F.round((stats["count"] / train.count() * 100), 2).alias("ratio(%)"),
    )

stats\
  .sort("label")\
  .show()

### Definir el modelo base para compración

Debido a que conocemos todas las etiquetas del conjunto de datos, podemos generar experimentos para conocer como se compara el algoritmo semi-supervisado contra su contraparte "normal".

En especifico podemos generar un "límite inferior", es decir, el desempeño cuando el algoritmo se entrena con poca data. Esperaríamos que el algoritmo semi-supervisado tenga un desempeño mejor a este límite.

De la misma forma, utilizando todas las etiquetas reales podemos generar un "límite superior". Esperamos que en el mejor de los casos el algoritmo semi-supervisado este alrededor de estos valores.

In [ ]:
#--definimos un objeto que obtenga el vector de atributos
feature_cols = train.columns
feature_cols.remove('output')
feature_cols.remove('label')

assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")

In [ ]:
#--transformamos nuestros datos de entrenamiento y prueba
train_features = assembler.transform(train)
test_features = assembler.transform(test)

In [ ]:
train_features.show(3)

In [ ]:
#--ejemplo del vector de atributos. Es un sparse vector.
train_features.select("features", "label").show(1, truncate = False)

In [ ]:
#--debido a que estaremos accediendo varias a veces a estos datos, los cargamos
# en memoria.
# La columna "output" la vamos a utilizar para medir métricas de desempeño al
# final.
train_features = train_features\
    .select("features", "output", "label")\
    .cache()

In [ ]:
#--dividimos el conjunto de datos en aquellos que tiene "etiqueta" y los que no.
labeled = train_features\
    .filter(train_features.label != 2.0)

unlabeled = train_features\
    .filter(train_features.label == 2.0)

### Límite inferior

In [ ]:
#--definimos un arbol de decisión
dt = DecisionTreeClassifier(maxDepth=5,
                            labelCol="label")
#--lo entrenamos solo con el approx. 2% de nuestros datos
model = dt.fit(labeled)

In [ ]:
#--realizamos predicciones en set de entrenamiento sin etiquetas y en el set
# de pruebas
pred_unlabeled = model\
    .transform(unlabeled)

pred_test = model\
    .transform(test_features)

In [ ]:
#--definimos el objeto que evaluara la predicciones
evaluator = MulticlassClassificationEvaluator(
    predictionCol="prediction",
    labelCol="output",
    metricName="accuracy",
)

In [ ]:
#--predicciones en set de entrenamiento sin etiqueta
temp = evaluator.evaluate(pred_unlabeled)
print(f'Precisión conjunto entrenamiento sin etiquetas: {temp:.3}')

In [ ]:
temp = evaluator.evaluate(pred_test)
print(f'Precisión conjunto de prueba: {temp:.3}')

### Límite superior

Utilizamos todos los datos etiquetados del conjunto de entrenamiento disponibles para darnos una idea de cuál sería el tope en desempeño que pudiera alcanzar el algoritmo semi-supervisado.

In [ ]:
#--cambiamos la columna de variable respuesta al modelo
dt.setLabelCol('output')

#--entrenamos el modelo
upperbound_model = dt.fit(train_features)

In [ ]:
#--realizamos las predicciones en el set de prueba
upperbound_pred = upperbound_model.transform(test_features)

In [ ]:
temp = evaluator.evaluate(upperbound_pred)
print(f'Precisión conjunto de prueba: {temp:.3}')

## Algoritmo semi-supervisado: diseño distribuido

Dado los supuestos de nuestro experimento:
* pocos datos con etiquetas (approx. 2%),
* muchos datos sin etiquetas,

al inicio el algoritmo tendrá una carga computacional cargada hacia la etapa de predicción y definición de las nuevas etiquetas. Al avanzar las iteraciones, tendremos más datos etiquetados por lo que el entrenamiento empezará a tener una carga más grande y las instancias con etiquetas tenderán a disminuir.

Si pensamos en una solución distribuida, debemos tener en consideración con la carga de trabajo pasa de un proceso al otro.

Al poner todos los datos dentro de un spark dataframe y utilizar un algoritmo de la MLlib, estamos asegurando que las dos fases __entrenamiento__ y __predicción__ se calculen de forma distriubida.

Dependiendo del tipo de problema, puede pasar que al inicio y al final tengamos muy pocas instancias para entrenar y para predecir respectivamente por lo que los recursos de spark pudieran ser demasiados para esos escenarios. Es un precio que podemos pagar por la ventaja de entrenar y predecir en modo distribuido.

La solución constará de los siguientes pasos:
1. Entrenar un modelo "global" usando solo las instancias que tienen etiquetas.
2. Predecir las instancias del conjunto de entrenamiento que no tiene etiqueta.
3. Decidir cuales son las predicciones que son suficientemente confiables para utilizarlas como instancias con etiqueta la siguiente iteración.

In [ ]:
#--observamos como el modelo que estamos usando nos regresa las predicciones.
# Notesé que "model" fue el modelo que entrenamos solo con el 2% de las
# instancias
preds = model\
    .transform(train_features)

preds\
    .select('prediction', 'probability')\
    .show(3, False)

Necesitamos elegir las predicciones que el clasificador considera son las más confiables para incorporarlas como etiquetas en la siguiente iteración.

Necesitamos obtener la predicción con mayor probabilidad del vector contenido en la columna "probability".

In [ ]:
#--separamos las columnas y escogemos la de mayor probabilidad
preds = preds.withColumn(
    "prob", F.array_max(vector_to_array("probability"))
  )

preds.show(3)

Tenemos que decidir el concepto de __predicciones confiables__. Para este ejemplo, dado que estamos utilizando árboles podemos definir __predicciones confiables__ como aquellas que sean igual a __1__.

Esta decisión depende del algoritmo y puede pasar que para otros algoritmos diferentes a los árboles proababilidad a 1 sea imposible de obtener. Este es un hyper-parámetro que tenemos que definir.

No podemos agregar a las etiquetas todas las instancias que obtuvieron una probabilidad de __1__ porque podemos sesgar el algoritmo hacia alguna clase. Por tanto tenemos que agregar las muestras en base a la distribución de etiquetas originales.

In [ ]:
#--observamos la distribución de las predicciones "confiables".
preds\
    .filter("prob == 1")\
    .groupBy("prediction")\
    .count()\
    .show()

El resultado anterior muestra que el clasificador tiene más confianza prediciendo la clase positiva que en el fenómeno original es precisamente lo opuesto, la clase con menos instancias.

Tenemos que definir una forma de asegurarnos que la selección de instancia a incorporarse como "etiquetas" tiene la misma distribución que el fenómeno original para cada una de las iteraciones.

In [ ]:
#--definimos una función que dada la distribución de clases del fenómeno y
# dado el número de instancias por clase con predicción igual a 1, nos regrese
# el número correcto de instancias a tomar como predicciones confiables de
# cada clase.

def instances_per_class(class_distrib, counts):
    # La función trabaja con respecto a la clase que esta menos representada

    index_min = np.argmin(counts / class_distrib)

    new_counts = counts[index_min] * (
        class_distrib / class_distrib[index_min]
    )

    return np.round(new_counts).astype("int")

Vamos a probar la función anterior. Para ello vamos a definir una función más que nos regrese la distribución de clases en el conjunto de datos que si tienen etiqueta

In [ ]:
def df_count(df, group_col):
    return np.array(
              df\
                .groupBy(group_col)\
                .count()\
                .sort(group_col)\
                .toPandas()["count"],
              dtype="int",
    )

In [ ]:
#--para probar la función anterior, calculamos la distribución del conjunto
# que si tiene etiquetas.
class_distrib = df_count(labeled, 'output')
class_distrib

In [ ]:
#--obtenemos la cuenta de las predicciones que son "confiables"
counts = df_count(preds.filter("prob == 1"), "prediction")
counts

In [ ]:
#--usamos la funcion instances_per_class para obtener el número de
# instancias correctas dada la distribución de clases de las etiquetas y el
# número de predicciones confiables

to_add = instances_per_class(class_distrib, counts)
to_add

Ya tenemos el número exacto de instancias que debemos agregar como etiquetas "reales" para la próxima iteración.

Para realizarlo vamos a agregar una nueva columna que incluya las etiquetas de la iteración anterior y las que deseamos agregar. Para crear esta columna notamos que las instancias nuevas que se incorporán como etiquetas reales,  cumplen con las siguientes características:

1. No tenían etiqueta, "label == 2"
2. Obtuvieron una predicción confiable, prob.==1
3. Seleccionadas de forma aleatoria de acuerdo a __to_add__

Desafortunadamente si deseamos cumplir con el número exacto de muestras a agregar vamos a incorporar complejidad computacional. Una forma simple de acerco es utilizar la misma idea que usamos para reducir el número de etiquetas al inicio, la desventaja es que no será exacta el número de instancias que agregemos.

Consideremos las siguientes proporciones:

In [ ]:
pcts = to_add / counts # elementwise operation
pcts

Para logralo, lo vamos a dividir en dos pasos. Primero vamos a agregar una columna extra que tenga el porcentaje que acabamos de calcular:

In [ ]:
#--columna donde ponemos el porcentaje de acuerdo a la clase
preds = preds.withColumn("pcts",
    F.when(preds.label == 0, pcts[0]).otherwise(pcts[1])
)

preds.show(5)

Con esta última columna tenemos todos los ingredientes para hacer la actualización de las nuevas etiquetas e incluir a las predicciones "confiables".

In [ ]:
#--actualizamos la variable respuesta.
# Si se cumplen las primeras tres condiciones se agrega una etiqueta que
# corresponde a una predicción confiable, sino se agregan las etiquetas de la
# iteración anterior que puede contener una instancia con etiqueta real o una
# instancia sin etiqueta.
preds = preds.withColumn("label",
    F.when(
        (preds.label == 2) &
        (preds.prob == 1) &
        (F.rand(seed=12345) <= preds.pcts),
        preds.prediction).otherwise(preds.label),
)

In [ ]:
#--rectificamos que tengamos un cambio en la distribución de etiquetas para
# comprobar que efectivamente tenemos más instancias con etiqueta
preds\
    .groupBy('label')\
    .count()\
    .sort('label')\
    .show()

Finalmente juntamos todos estos ingredientes en una sola función:

In [ ]:
def self_training(train_features, max_iter=10, seed=12345,
                  num_classes=2, max_depth=5):

    # definimos el modelo que vamos a utilizar
    dt = DecisionTreeClassifier(maxDepth=max_depth, labelCol="label")

    # obtenemos la distribución de clases de las muestras con etiquetas
    class_distrib = df_count(train_features.filter("label != 2"), "label")

    for i in range(max_iter):
        labeled = train_features.filter("label != 2")

        print(f"Iteration: {i} - labeled size: {labeled.count()}")

        # Entrenamiento
        model = dt.fit(labeled)

        # Predecir y obtener probabilidades
        preds = model.transform(train_features)
        preds = preds.withColumn("prob",
            F.array_max(vector_to_array("probability"))
        )

        # numero de muestras que podriamos incorporar
        counts = df_count(
            preds.filter("label == 2 AND prob == 1"), "prediction"
        )

        # ponemos una condición para detener el ciclo si no hay instancias
        # a sustituir o si no hay instancias de todas las clases
        if 0 in counts or len(counts) < num_classes:
            break

        to_add = instances_per_class(class_distrib, counts)
        print(f"Adding {to_add[0]} instances from class 0, "
              f"and {to_add[1]} from class 1")

        # calculamos el porcentaje de instancias a incluir
        pcts = to_add / counts

        preds = preds.withColumn("pcts",
            F.when(preds.label == 0, pcts[0]).otherwise(pcts[1]),
        )

        preds = preds.withColumn("label",
            F.when(
                (preds.label == 2) & (preds.prob == 1)
                & (F.rand(seed=12345) <= preds.pcts),
                preds.prediction).otherwise(preds.label),
        )

        train_features = preds.select("features", "output", "label").cache()

    return train_features.filter("label != 2")

In [ ]:
# reset train_features
train_features = assembler.transform(train)
%time enlarged_labeled = self_training(train_features)

In [ ]:
print(f'Número de instancias del conjunto de datos extendido: {enlarged_labeled.count()}')

Con el nuevo conjunto de datos podemos entrenar nuestro modelo y medir su desempeño!!

In [ ]:
model = dt.fit(enlarged_labeled)
preds_trans = model.transform(unlabeled)
preds_inductive = model.transform(test_features)

print(f"Precisión en las instancias sin etiquetas del conjunto de entrenamiento = {evaluator.evaluate(preds_trans)}")
print(f"Precisión en el conjunto de prueba = {evaluator.evaluate(preds_inductive)}")